In [1]:
print('Suraj Singh')

Suraj Singh


In [2]:
print('Babu Singh');

Babu Singh


In [3]:
%pwd

'c:\\Users\\suraj\\OneDrive\\Desktop\\ML Chatbot\\ML-Chatbot\\research'

In [4]:
import os
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\suraj\\OneDrive\\Desktop\\ML Chatbot\\ML-Chatbot'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain'